# DNBLab Jupyter Notebook Tutorial

## OAI(2) - Schnittstellenabfragen, Datenauslieferung und Ergebnisanzeige

Diese DNBLab-Tutorials beschreiben Beispielabfragen über die OAI2- und OAI-Schnittstelle mit Python. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden. Die Tutorials umfassen die exemplarische Anfrage und Ausgabe der Daten in den verschiedenen Metadatenformaten zur weiteren Verarbeitung.

**Inhaltsverzeichnis:** 


* [Einrichten der Arbeitsumgebung](#API1)
* [OAI2 - Schnittstelle ](#API2)
* [OAI - Schnittstelle ](#API3)

## Einrichten der Arbeitsumgebung <a class="anchor" id="API1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Bibliotheken importiert werden. Für Anfragen über die OAI(2)-Schnittstellen wird sickle https://sickle.readthedocs.io/en/latest/tutorial.html und zur Verarbeitung der XML-Daten etree https://docs.python.org/3/library/xml.etree.elementtree.html verwendet. Mit dem OAIRecordReader https://polymatheia.readthedocs.io/en/latest/api/polymatheia_data_reader.html können Elemente aus dem METS/MODS Format ausgelesen werden.

In [ ]:
from sickle import Sickle
from lxml import etree
from polymatheia.data.reader import OAIRecordReader

## OAI2- Schnittstelle <a class="anchor" id="API2"></a>

Inhaltsverzeichnis:

* [1. Grundlegende Funktionen](#Teil1)
* [2. Ausgabe von Titeln eines Projekts ab bestimmtem Zeitraum](#Teil2)
* [3. Zusätzliche Ausgabe der URNs/Links zu den Objekten](#Teil3)
* [4. Ausgabe der gescannten Bilder](#Teil4)




### 1. Grundlegende Funktionen <a class="anchor" id="Teil1"></a>

Die Daten der frei verfügbaren digitalisierten Objekte werden über die OAI2-Schnittstelle standardmäßig als XML-Antwort in dem Format METS/MODS ausgeliefert. Zusätzlich steht die Auslieferung der Daten in Dublin Core (DC) zur Verfügung.

Die OAI2 Basis URL ist http://services.dnb.de/oai2/repository. 

In [ ]:
sickle = Sickle('http://services.dnb.de/oai2/repository')

Im nächsten Schritt werden alle verfügbaren Objektsammlungen (Projekte) mit der Funktion ListSets abgefragt und angezeigt.

In [ ]:
oai_sets = sickle.ListSets()
for oai_set in oai_sets:
    print('setSpec value for selective harvesting: ' + oai_set.setSpec)
    print('Name of the set (setName): ' + oai_set.setName + '\n')

Die bisher zur Verfügung stehenden freien Objektsammlungen entsprechen folgenden Projekten (oai_sets):
 
| oai_set | Beschreibung | Anzeige im Katalog |
| --- | --- | --- |
| dnb:digitalisate-oa:projekt4 | 100.000 Seiten aus 30 Titeln, die ein breites Spektrum der Exilpresse 1933-1945 repräsentieren | <a href="https://portal.dnb.de/opac.htm?method=showPreviousResultSite&currentResultId=%22exilpresse%22+and+%22digital%22%26any%26dnb.dea.exp%26leipzig&currentPosition=10">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt7 | Gemeinfreie Digitalisate der Exilsammlungen | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd007+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt8 | 1Mehr als 1.200 Bücher, Broschüren, Plakate und andere Einblattdrucke zum Weltkrieg 1914-1918 | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd008+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt26 | Mehr als 600 digitalisierte Quellen zur deutschen Revolution 1918/1919 und zur Nachkriegszeit | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd026+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt28 | Digitalisierte Werke aus Architektur, Design, Malerei und Druckkunst von 1918 bis 1933 | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd028+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt29 | Mehr als 2.400 Bildnisse von Buchhändlern, Buchdruckern und Verlegern des 17. bis 20. Jahrhunderts | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd029+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>

Die verfügbaren Metadatenformate können mit der Funktion ListMetadataFormats abgefragt und mit print() ausgegeben werden. 


In [ ]:
oai_formats = sickle.ListMetadataFormats()
for oai_format in oai_formats:
    print(oai_format.metadataPrefix)

### 2. Ausgabe von Titeln eines Projekts in bestimmten Zeitraum <a class="anchor" id="Teil2"></a>

Um beispielhaft alle Datensätze der Buchhändlerportraits (Projekt29) ab dem 01.04.2019 aufzulisten, müssen für die Abfrage Metadatenformat, Zeitraum und Projekt angegeben werden.
Die OAI Funktionen werden unter http://www.openarchives.org/OAI/2.0/ spezifiziert und auf der DNBLab Seite unter Funktionen beschrieben.
Die Daten befinden sich in der XML-Antwort als Inhalt zwischen beschreibenden <Tags>. Für die gezielte Abfrage der gewünschten Informationen aus dem Metadatenformat DC können bei Bedarf die DC-Spezifikationen http://www.openarchives.org/OAI/2.0/oai_dc/ und http://purl.org/dc/elements/1.1/ herangezogen werden.


In [ ]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count=0

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc', 'from': '2019-04-30', 'set': 'dnb:digitalisate-oa:projekt29'}):
    
    if ('ger' in record.raw):
        #print('-' + record.raw + '-')
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if (result):
            count += 1
            author = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:creator/text()', namespaces=namespaces)
            print(str(count) + ": " + result[0])
            if author:
                print(author[0])

### 3. Zusätzliche Ausgabe der Links zu den Objekten <a class="anchor" id="Teil3"></a>

Damit zusätzlich zu den Titeln die Links zu den Objekten ausgegeben werden, wird der URN entsprechend des dc-Schemas /oai:record/oai:metadata/oai_dc:dc/dc:identifier/text() angesprochen und zusammen mit dem Titel durchnummeriert angezeigt. 

In [ ]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count=0

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc', 'from': '2019-04-30', 'set': 'dnb:digitalisate-oa:projekt29'}):
    
    if ('ger' in record.raw):
        #print('-' + record.raw + '-')
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if (result):
            count += 1
            urn = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:identifier/text()', namespaces=namespaces)
            print(str(count) + ": " + result[0])
            if urn:
                print(urn[0])

### 4. Ausgabe der gescannten Bilder <a class="anchor" id="Teil4"></a>

Das Format METS bildet neben den bibliographischen Daten im Format MODS die Struktur der digitalen Objekte bzw. die Reihenfolge der gescannten Bilder ab. Für die gezielte Abfrage der gewünschten Informationen aus dem Metadatenformat METS können bei Bedarf die METS-Spezifikationen http://www.loc.gov/METS/ und https://www.loc.gov/standards/mets/mets.xsd herangezogen werden. Die Links der einzelnen Bilder (Thumbnails und Defaultbilder) können über <xsd:element name="fileGrp"> angesprochen und die ersten 10 zur weiteren Verarbeitung ausgegeben werden.

In [2]:
reader = OAIRecordReader('http://services.dnb.de/oai2/repository', max_records=10, metadata_prefix='mets', set_spec='dnb:digitalisate-oa:projekt29')

for record in reader:

    if isinstance(record.metadata['{http://www.loc.gov/METS/}mets'].METS_fileSec.METS_fileGrp, list):

        print(record.metadata['{http://www.loc.gov/METS/}mets'].METS_fileSec.METS_fileGrp[0].METS_file.METS_FLocat._attrib.xlink_href)

    else:

        print(record.metadata['{http://www.loc.gov/METS/}mets'].METS_fileSec.METS_fileGrp.METS_file)

https://portal.dnb.de/bookviewer/view/1175640522/img/page/1/p.jpg?reduce=51.5
https://portal.dnb.de/bookviewer/view/1163108812/img/page/1/p.jpg?reduce=48.2
https://portal.dnb.de/bookviewer/view/1163108804/img/page/1/p.jpg?reduce=46.9
https://portal.dnb.de/bookviewer/view/1163108790/img/page/1/p.jpg?reduce=62.8
https://portal.dnb.de/bookviewer/view/1163108782/img/page/1/p.jpg?reduce=62.3
https://portal.dnb.de/bookviewer/view/1163108774/img/page/1/p.jpg?reduce=38.3
https://portal.dnb.de/bookviewer/view/1163108766/img/page/1/p.jpg?reduce=37.5
https://portal.dnb.de/bookviewer/view/1163108758/img/page/1/p.jpg?reduce=66.3
https://portal.dnb.de/bookviewer/view/116310874X/img/page/1/p.jpg?reduce=54.4
https://portal.dnb.de/bookviewer/view/1163108731/img/page/1/p.jpg?reduce=52.7


## OAI- Schnittstelle <a class="anchor" id="API4"></a>

Inhaltsverzeichnis:

* [1. Grundlegende Funktionen](#Teil5)
* [2. Ausgabe von Titeln in bestimmten Zeitraum](#Teil6)
* [3. Zusätzliche Ausgabe der Links zu den Objekten](#Teil7)

### 1. Grundlegende Funktionen <a class="anchor" id="Teil5"></a>

Die Daten werden über die OAI-Schnittstelle standardmäßig als XML-Antwort in verschiedenen Formaten ausgeliefert.

Die OAI Basis URL ist http://services.dnb.de/oai/repository.

In [ ]:
sickle = Sickle('http://services.dnb.de/oai/repository')

Im nächsten Schritt werden alle verfügbaren Datensets (Kataloge) mit der Funktion ListSets abgefragt und angezeigt.

In [ ]:
oai_sets = sickle.ListSets()
for oai_set in oai_sets:
    print('setSpec value for selective harvesting: ' + oai_set.setSpec)
    print('Name of the set (setName): ' + oai_set.setName + '\n')

Hier eine Auswahl der über OAI verfügbaren Titeldaten (oai_sets):

| Auswahl | Wert für Parameter "set" | 
| --- | --- | 
| Deutsche Nationalbibliografie ohne Gemeinsame Normdatei| dnb:wv (nur Datensätze nach abgeschlossener Bearbeitung) | 
| Deutsche Nationalbibliografie, Reihe A (Publikationen des Verlagsbuchhandels) | dnb:reiheA (inkl. Datensätze in Bearbeitung und abgeschlossene Bearbeitung) |
| Deutsche Nationalbibliografie, Reihe B (Publikationen außerhalb des Verlagsbuchhandels) | dnb:reiheB (inkl. Datensätze in Bearbeitung und abgeschlossene Bearbeitung)
dnb:wv:reiheB (nur Datensätze nach abgeschlossener Bearbeitung) |
| Deutsche Nationalbibliografie, Reihe H (Hochschulschriften)| dnb:reiheH (inkl. Datensätze in Bearbeitung und abgeschlossene Bearbeitung)
dnb:wv:reiheH (nur Datensätze nach abgeschlossener Bearbeitung) |
| Deutsche Nationalbibliografie, Reihe O (Online-Publikationen) | dnb:reiheO (inkl. Datensätze in Bearbeitung und abgeschlossene Bearbeitung)
dnb:wv:reiheO (nur Datensätze nach abgeschlossener Bearbeitung) |
| Online-Publikationen ohne Einschränkungen | dnb-all:online |
| Deutsche Nationalbibliografie: Digitalisierte Inhaltsverzeichnisse| dnb:toc |

Die verfügbaren Metadatenformate können mit der Funktion ListMetadataFormats abgefragt und mit print() ausgegeben werden.

In [ ]:
oai_formats = sickle.ListMetadataFormats()
for oai_format in oai_formats:
    print(oai_format.metadataPrefix)

### 2. Ausgabe von Titeln eines Projekts in bestimmten Zeitraum <a class="anchor" id="Teil6"></a>
Um beispielhaft Online-Dissertationen aus der Sachgruppe „Sozialwissenschaften, Soziologie, Anthropologie“ ab dem 01.04.2019 aufzulisten, müssen für die Abfrage Metadatenformat, Zeitraum und Projekt angegeben werden. Die OAI Funktionen werden unter http://www.openarchives.org/OAI/2.0/ spezifiziert und auf der DNBLab Seite unter Funktionen beschrieben. Die Daten befinden sich in der XML-Antwort als Inhalt zwischen beschreibenden . Für die gezielte Abfrage der gewünschten Informationen aus dem Metadatenformat DC, können bei Bedarf die DC-Spezifikationen http://www.openarchives.org/OAI/2.0/oai_dc/ und http://purl.org/dc/elements/1.1/ herangezogen werden.

In [ ]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count=0

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc', 'from': '2019-04-30', 'set': 'dnb-all:online:dissertations:sg300'}):
    
    if ('ger' in record.raw):
        #print('-' + record.raw + '-')
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if (result):
            count += 1
            author = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:creator/text()', namespaces=namespaces)
            print(str(count) + ": " + result[0])
            if author:
                print(author[0])

### 3. Zusätzliche Ausgabe der Links zu den Objekten <a class="anchor" id="Teil7"></a>
Damit zusätzlich zu den Titeln die Links zu den Objekten ausgegeben werden, wird der URN entsprechend des dc-Schemas /oai:record/oai:metadata/oai_dc:dc/dc:identifier/text() angesprochen und zusammen mit dem Titel durchnummeriert angezeigt.

In [ ]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count=0

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc', 'from': '2019-04-30', 'set': 'dnb-all:online:dissertations:sg300'}):
    
    if ('ger' in record.raw):
        #print('-' + record.raw + '-')
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if (result):
            count += 1
            urn = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:identifier/text()', namespaces=namespaces)
            print(str(count) + ": " + result[0])
            if urn:
                print(urn[0])